In [1]:
import awkward as ak
import uproot
import numpy as np

In [2]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema

In [3]:
signaldomain = '/ecoderemdvol/econ_ntuples/qk/econ_ntuple_40_particles.root'
signalfile = uproot.open(signaldomain)

In [4]:
events = NanoEventsFactory.from_root(
    signalfile,
    treepath='FloatingpointThreshold0DummyHistomaxGenmatchGenclustersntuple/HGCalTriggerNtuple',
    #entry_stop=10,
    #metadata={"dataset": "llp"},
    schemaclass=BaseSchema,
).events()

In [5]:
gen_pt = events.gen_pt

In [6]:
mask = gen_pt < 35

In [7]:
mask = mask[:,1]

In [8]:
mask

<Array [False, False, False, ... False, False] type='20 * bool'>

In [9]:
tc_mask_data = ak.Array(
    {
        'tc_zside': events['tc_zside'][mask],
        'tc_layer': events['tc_layer'][mask],
        'tc_waferu': events['tc_waferu'][mask],
        'tc_waferv': events['tc_waferv'][mask],
        'tc_cellu': events['tc_cellu'][mask],
        'tc_cellv': events['tc_cellv'][mask],
        'tc_data': events['tc_data'][mask],
        'tc_simenergy': events['tc_simenergy'][mask],
        'tc_eta': events['tc_eta'][mask]
    }                
)

In [10]:
df = ak.to_pandas(tc_mask_data)

In [11]:
import pandas as pd

In [12]:
dfRemap = pd.read_csv('TrainingData/tcRemap.csv')
df = df.reset_index().merge(dfRemap)

df['ADCT'] = (df.tc_data* ((1./np.cosh(df.tc_eta))*2**12).astype(int)/2**12).astype(int)

#create new unique index (can't get pivot working on multi-indexing, but this is essentially the same)
df['WaferEntryIdx'] = (df.entry*1000000 + df.tc_layer*10000 + df.tc_waferu*100 + df.tc_waferv)*df.tc_zside

val = 'ADCT'
dfTrainData = df.pivot_table(index='WaferEntryIdx',columns='tc_cell_train',values=val).fillna(0).astype(int)
dfTrainData.columns = [f'CALQ_{i}' for i in range(48)]

dfTrainData[['entry','zside','layer','waferu','waferv']] = df.groupby(['WaferEntryIdx'])[['entry','tc_zside','tc_layer','tc_waferu','tc_waferv']].mean()

dfTrainData['simenergy'] = df.groupby(['WaferEntryIdx'])[['tc_simenergy']].sum()

#Mapping wafer_u,v to physical coordinates
dfEtaPhi=pd.read_csv('TrainingData/WaferEtaPhiMap.csv')
dfTrainData=dfTrainData.merge(dfEtaPhi, on=['layer','waferu','waferv'])
dfTrainData.reset_index(drop=True,inplace=True)

In [13]:
dfTrainData

,CALQ_0,CALQ_1,CALQ_2,CALQ_3,CALQ_4,CALQ_5,CALQ_6,CALQ_7,CALQ_8,CALQ_9,...,CALQ_47,entry,zside,layer,waferu,waferv,simenergy,subdet,tc_eta,tc_phi
0,0,0,0,0,0,0,0,0,0,0,...,0,0.0,-1.0,50.0,6.0,7.0,0.0,2,2.379769,2.002501
1,0,0,0,0,0,0,0,0,0,0,...,0,0.0,-1.0,50.0,6.0,6.0,0.0,2,2.402795,2.174989
2,0,0,0,0,0,0,0,0,0,0,...,0,0.0,-1.0,50.0,5.0,4.0,0.0,2,2.692211,2.404243
3,0,0,0,0,0,0,0,0,0,0,...,0,0.0,-1.0,50.0,5.0,-1.0,0.0,2,2.371235,-2.885253
4,0,0,5,0,0,0,0,0,0,0,...,0,0.0,-1.0,50.0,4.0,5.0,0.0,2,2.754663,1.986027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13517,0,0,0,0,0,0,0,0,0,0,...,0,0.0,1.0,50.0,3.0,5.0,0.0,2,2.843674,1.717921
13518,0,0,0,0,0,0,0,0,0,0,...,0,0.0,1.0,50.0,4.0,-1.0,0.0,2,2.567744,-2.809976
13519,0,0,0,0,0,0,0,0,0,0,...,0,0.0,1.0,50.0,4.0,0.0,0.0,2,2.748377,-2.975534
13520,0,0,0,0,0,0,0,0,0,0,...,0,0.0,1.0,50.0,4.0,6.0,0.0,2,2.621211,1.829199


In [16]:
13522/265202

0.050987549113505934